In [1]:
import os
import re
import string
import pandas as pd
import cleantext
from tqdm import tqdm
import langdetect

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [24]:
DATA_PATH = "../data"

CHAR_MAP = {
    "â": "a",
    "İ": "i",
    "I": "ı",
    "Â": "a",
    "Î": "i",
    "î": "i",
    "ì": "i",
    "í": "i",
    "û": "u",
    "Û": "u", 
    "|": "",
    "¦": "",
    "~": "",
    "°": "",
    "^": "",
    "+": "",
    "<": "",
    "=": "",
    ">": "",
    "±": "",
    "ĝ": "ğ",
    "ܧ": "",
    "'ާ": "",
    "ޱ": "",
    "绫": "",
    "ȥ": "",
    "ǥ": "",
    ". com. tr": ".com.tr",
    ". com": ".com",
    " km.": "km",
    "@hotmail. com": "@hotmail.com",
    " hotmail. com": "@hotmail.com"
}

EMAIL_PATTERNS = [
    "gmail",
    "hotmail",
    "yahoo",
    ".com",
]

In [25]:
class Cleaner():
    def __init__(self):
        pass
    
    def __clean_text(self, text, clean_punc):
        for char, replaced_char in CHAR_MAP.items():
            text = text.replace(char, replaced_char) 
        text = cleantext.clean(
            text,
            to_ascii=False,
            no_line_breaks=True,
            no_urls=True,
            no_emails=True,
            no_phone_numbers=True,
            no_numbers=True,
            no_digits=True,
            no_currency_symbols=True,
            no_punct=clean_punc,
            no_emoji=True,
            replace_with_url=' ',
            replace_with_email=' ',
            replace_with_phone_number=' ',
            replace_with_punct='',
            replace_with_number='',
            replace_with_digit='',
            replace_with_currency_symbol=''
        )
        text = " ".join([token for token in text.split() if len(token) > 1])
        if not clean_punc:
            for email_token in EMAIL_PATTERNS:
                text = " ".join([token for token in text.split() if email_token not in token])
        return text
    
    def __remove_non_turkish_contents(self, df):
        df = df[df["content"] != ""]
        df["suggested_lang"] = df["content"].apply(lambda content: langdetect.detect(content))
        df = df[df["suggested_lang"] == "tr"]
        df = df.drop("suggested_lang", axis=1)
        return df
    
    def __split_sentences(self, text):
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
        return sentences
    
    def __remove_punctuations(self, sentences):
        sentences = [self.__clean_text(sent, clean_punc=True) for sent in sentences]
        return sentences
    
    def __clean_bad_sentences(self, sentences):
        sentences = [sent for sent in sentences if len(sent.split()) > 1 and len(sent) >= 5]
        return sentences
    
    def process_df(self, df):
        for col in ["pub_name", "title", "content"]:
            clean_punc = False if col == "content" else True
            df[col] = df[col].apply(lambda text: self.__clean_text(text, clean_punc))
        df = self.__remove_non_turkish_contents(df)
        df["sentences"] = df["content"].apply(lambda text: self.__split_sentences(text))
        df["sentences"] = df["sentences"].apply(lambda sentences: self.__remove_punctuations(sentences))
        df["sentences"] = df["sentences"].apply(lambda sentences: self.__clean_bad_sentences(sentences))
        return df

In [26]:
text_cleaner = Cleaner()

In [27]:
df = pd.read_excel("../../data/Rusya-Ukrayna Savaşı NS İçerikleri.xlsx")

In [28]:
df_cleaned = text_cleaner.process_df(df.copy())

In [29]:
df_cleaned

,date,pub_name,type,title,content,Label,sentences
0,28 Şubat 2022 Pazartesi,pusula haber konya,ulusal,rus işgali mutlaka durdurulmalı,rus işgali mutlaka durdurulmalı! konytfde kemh...,hate,[rus işgali mutlaka durdurulmalı konytfde kemh...
1,2022-03-02 00:00:00,milat,ulusal,ruslar barbarca saldırıyor,uydu görüntüleri yüzlerce zırhlı araç ve topta...,hate,[uydu görüntüleri yüzlerce zırhlı araç ve topt...
2,2022-03-03 00:00:00,türkiyede yeniçağ,ulusal,rus yayılmacılığı putinle hortladı,"rus yayılm acidkı putin'le hrtadı akşener, ukr...",hate,[rus yayılm acidkı putinle hrtadı akşener ukra...
3,2022-03-03 00:00:00,ilkses gazetesi,ulusal,harkivde rusların ilerlemesi sürüyor,harkiv'de rusların ilerlemesi sürüyor rusya'nı...,hate,[harkivde rusların ilerlemesi sürüyor rusyanın...
4,2022-03-06 00:00:00,akşam,ulusal,ruslar kievde ingiliz gazetecilere ateş açtı,"ruslar, kiev'de ingiliz gazetecilere ateş açtı...",hate,[ruslar kievde ingiliz gazetecilere ateş açtı ...
5,2022-03-06 00:00:00,bizim anadolu gazetesi,ulusal,ruslar ilerliyor ukrayna direniyor,ruslar ilerliyor ukrayna direniyor rusya'nın u...,hate,[ruslar ilerliyor ukrayna direniyor rusyanın u...
6,07 Mart 2022 Pazartesi,türkgün,ulusal,ruslar starokostiantyniv üssünü vurdu,"ruslar, starokostiantyniv üssü'nü vurdu i\ rus...",hate,[ruslar starokostiantyniv üssünü vurdu rusya u...
7,2022-03-08 00:00:00,diriliş postası,ulusal,direnme teslim ol ukrayna,"murat özer murat direnme, teslim ol ukrayna! b...",hate,[murat özer murat direnme teslim ol ukrayna bü...
8,2022-03-17 00:00:00,korkusuz,ulusal,ruslar çocukları hastanede esir aldı,ruslar cocukları hastanede esir aldı ukrayna'n...,hate,[ruslar cocukları hastanede esir aldı ukraynan...
9,14 Mart 2022 Pazartesi,gazete pencere,ulusal,ruslar abdli gazeteciyi öldürdü,ruslar abd'li gazeteciyi öldürdü rusya'nın kuş...,hate,[ruslar abdli gazeteciyi öldürdü rusyanın kuşa...


In [31]:
alphabet = set()
for text in df_cleaned["sentences"].values:
    for sentence in text:
        alphabet = set.union(alphabet, set(sentence))

In [32]:
alphabet

{' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'ç',
 'ö',
 'ü',
 'ğ',
 'ı',
 'ş'}

In [35]:
df_cleaned = df_cleaned.reset_index().rename(columns={"index": "id"})
df_cleaned["phase"] = "test"

In [36]:
df_cleaned.to_csv('../../data/data_cleaned_sentences_ukrayna.csv', sep='|', index=False)